# League of Legends NN

## Loading Libaries

In [1]:
# Importing Libraries



# Basic Libraries #################################################################################
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 
import seaborn as sns
import datetime
import time


# For Feature Engineering #########################################################################

# For Plotting Correlation Matrix and Scatter Plot
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler

# Min Max Scaler
from sklearn.preprocessing import MinMaxScaler

# One Hot Encoding
from sklearn.preprocessing import OneHotEncoder

# Label Encoder
from sklearn.preprocessing import LabelEncoder


# For Machine Learning Techniques #################################################################################

# For Data Anaylsis
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


# Personal Preference #############################################################################
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Loading Tensorflow


import tensorflow as tf
from tensorflow import keras

# Load the TensorBoard notebook extension
%load_ext tensorboard

# Clear any logs from previous runs
!rm -rf ./logs/ 

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
# Select Dataset File and Target Variable


dataset_file = "base_def.csv"

target_variable = 'Win'

## Preliminary Data Analysis

In [4]:
# Uploading dataset as a Pandas file


df_pd = pd.read_csv(dataset_file)
df_pd.head(30)

FileNotFoundError: [Errno 2] No such file or directory: 'base_def.csv'

In [5]:
# One-Hot Encoding


df_pd2 = pd.get_dummies(df_pd, drop_first = True, columns = ['Team','Position','Win'])
df_pd2.head()

,Unnamed: 0,Champ,GoldEarned,Kills,Deaths,Assists,totalDamageDealtToChampions,damageDealtToTurrets,visionScore,totalDamageTaken,...,WRgeneral,WRchamp,Duration_secs,Role,Team_200,Position_JUNGLE,Position_MIDDLE,Position_NONE,Position_TOP,Win_True
0,1,Garen,10913,3,13,6,14015,0,24,38262,...,0.6000,0.5556,2157,SOLO,0,0,0,0,1,0
1,2,Irelia,11004,10,3,3,13895,6226,10,14749,...,0.8000,0.8824,1216,SOLO,0,0,1,0,0,1
2,3,Kayn,13191,10,8,15,29583,2259,18,48254,...,0.5667,0.8333,1949,NONE,0,1,0,0,0,1
3,4,Tristana,7025,2,6,3,11439,793,21,15039,...,0.4000,0.0000,1449,DUO_CARRY,0,0,0,0,0,0
4,5,Jhin,10867,4,4,8,13623,800,12,11402,...,0.4667,1.0000,1675,DUO_CARRY,1,0,0,0,0,1


In [6]:
# Manually Changing Variables 


# Adding Gold Earned per Minute Variable
df_pd2['Gold_Earned_per_Minute'] = df_pd2.GoldEarned//(df_pd2.Duration_secs//60)

# Dropping the Unamed Variable since it is equalivant to the index
df_pd2 = df_pd2.drop(['Unnamed: 0'], axis = 1)

# Renaming Columns
df_pd2 = df_pd2.rename(columns={"Team_200": "Team 2", "Position_JUNGLE": "Jungle", "Position_MIDDLE": "Mid", 
                                "Position_NONE": "Support", "Position_TOP": "Top","Win_True": "Win"})

df_pd2.head()

,Champ,GoldEarned,Kills,Deaths,Assists,totalDamageDealtToChampions,damageDealtToTurrets,visionScore,totalDamageTaken,turretKills,...,WRchamp,Duration_secs,Role,Team 2,Jungle,Mid,Support,Top,Win,Gold_Earned_per_Minute
0,Garen,10913,3,13,6,14015,0,24,38262,0,...,0.5556,2157,SOLO,0,0,0,0,1,0,311
1,Irelia,11004,10,3,3,13895,6226,10,14749,3,...,0.8824,1216,SOLO,0,0,1,0,0,1,550
2,Kayn,13191,10,8,15,29583,2259,18,48254,0,...,0.8333,1949,NONE,0,1,0,0,0,1,412
3,Tristana,7025,2,6,3,11439,793,21,15039,0,...,0.0000,1449,DUO_CARRY,0,0,0,0,0,0,292
4,Jhin,10867,4,4,8,13623,800,12,11402,0,...,1.0000,1675,DUO_CARRY,1,0,0,0,0,1,402


In [7]:
# String Indexer


df_SI = df_pd2

cat_columns = df_SI.select_dtypes(['object'])
cat_columns

#Convert from object type to cat. type
for i in cat_columns:
    df_SI[i] = df_SI[i].astype('category')

# Convert to Encoded values
cat_columns = df_SI.select_dtypes(['category'])
LE = LabelEncoder()
for i in cat_columns:
    df_SI[i] = LE.fit_transform(df_SI[i])
    
df_pd4 = df_SI
df_pd4

,Champ,GoldEarned,Kills,Deaths,Assists,totalDamageDealtToChampions,damageDealtToTurrets,visionScore,totalDamageTaken,turretKills,...,WRchamp,Duration_secs,Role,Team 2,Jungle,Mid,Support,Top,Win,Gold_Earned_per_Minute
0,33,10913,3,13,6,14015,0,24,38262,0,...,0.5556,2157,4,0,0,0,0,1,0,311
1,40,11004,10,3,3,13895,6226,10,14749,3,...,0.8824,1216,4,0,0,1,0,0,1,550
2,55,13191,10,8,15,29583,2259,18,48254,0,...,0.8333,1949,3,0,1,0,0,0,1,412
3,120,7025,2,6,3,11439,793,21,15039,0,...,0.0000,1449,1,0,0,0,0,0,0,292
4,46,10867,4,4,8,13623,800,12,11402,0,...,1.0000,1675,1,1,0,0,0,0,1,402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23747,100,3770,2,6,2,3526,804,7,7750,0,...,0.5000,997,2,0,0,0,1,0,0,235
23748,49,10314,6,3,6,14162,2617,35,14083,3,...,0.5000,1357,1,1,0,0,0,0,1,468
23749,31,12067,8,0,11,13767,5288,20,15809,0,...,1.0000,1433,4,1,0,1,0,0,1,524
23750,11,9815,5,0,19,11807,3559,45,10737,1,...,0.6667,1525,1,1,0,0,0,0,1,392


## Machine Learning Techniques

In [8]:
# Splitting the Data


df_datasplit = df_pd2

X = df_datasplit.drop(target_variable, axis = 1)
X = np.asarray(X).astype('float32')
y = df_datasplit.pop(target_variable)
y = np.asarray(y).astype('float32')

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2, random_state = 123)

In [9]:
# Designing the Model


model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(16, input_dim=26, activation=tf.nn.relu)) #Input Layer
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu)) #Hidden Layer 1
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)) #Output Layer

In [10]:
# Setting up how the model runs


model.compile(optimizer='adam', #Could Plug-in Stocastic Gradient Descent
             loss='mse', #categorical_crossentropy is the go-to
             metrics=['accuracy'])

In [11]:
# Logging Changes



log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [12]:
# Training the Model


model.fit(x=X_train,
          y=y_train,
          epochs=10,
         validation_data=(X_test, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/10
594/594 [==============================] - 1s 888us/step - loss: 0.4965 - accuracy: 0.5035 - val_loss: 0.4913 - val_accuracy: 0.5087
Epoch 2/10
594/594 [==============================] - 0s 715us/step - loss: 0.4964 - accuracy: 0.5036 - val_loss: 0.4913 - val_accuracy: 0.5087
Epoch 3/10
594/594 [==============================] - 0s 700us/step - loss: 0.4964 - accuracy: 0.5036 - val_loss: 0.4913 - val_accuracy: 0.5087
Epoch 4/10
594/594 [==============================] - 0s 709us/step - loss: 0.4964 - accuracy: 0.5036 - val_loss: 0.4913 - val_accuracy: 0.5087
Epoch 5/10
594/594 [==============================] - 0s 703us/step - loss: 0.4964 - accuracy: 0.5036 - val_loss: 0.4913 - val_accuracy: 0.5087
Epoch 6/10
594/594 [==============================] - 0s 740us/step - loss: 0.4964 - accuracy: 0.5036 - val_loss: 0.4913 - val_accuracy: 0.5087
Epoch 7/10
594/594 [==============================] - 0s 732us/step - loss: 0.4964 - accuracy: 0.5036 - val_loss: 0.4913 - val_accuracy:

In [13]:
# Testing the model


val_loss, val_acc = model.evaluate(X_test, y_test)
print(val_loss, val_acc)

149/149 [==============================] - 0s 424us/step - loss: 0.4913 - accuracy: 0.5087
0.4912649989128113 0.5087350010871887


In [14]:
# Plotting Accuracy

%tensorboard --logdir logs/fit


Reusing TensorBoard on port 6006 (pid 23868), started 0:07:02 ago. (Use '!kill 23868' to kill it.)

In [15]:
loss_train = history.history['train_loss']

loss_val = history.history['val_loss']

epochs = range(1,35)

plt.plot(epochs, loss_train, 'g', label='Training loss')

plt.plot(epochs, loss_val, 'b', label='validation loss')

plt.title('Training and Validation loss')

plt.xlabel('Epochs')

plt.ylabel('Loss')

plt.legend()

plt.show()

NameError: name 'history' is not defined